In [19]:
import pandas as pd

In [3]:
# load data
data = pd.read_csv('~/Desktop/final_data_v0-2.csv',  dtype=str)

In [4]:
data.head(100)

,PBMVendor,PharmacyID,DrugShortName,UnitCost,PharmacyName,PharmacyStreetAddress1,PharmacyCity,PharmacyZip,PharmZip,DrugLabelName
0,MedImpact,59ROCKINGHAMST03038,HYDROCHLOROTHIAZIDE,0.3333333333333333,RITE AID PHARMACY 03310,59 ROCKINGHAM ST,DERRY ...,03038,030,HYDROCHLOROTHIAZIDE
1,MedImpact,718SMYTHRD03104,GABAPENTIN,0.20355555555555555,MANCHESTER VAMC PHARMACY,718 SMYTH RD,MANCHESTER ...,03104,031,GABAPENTIN
2,MedImpact,718SMYTHRD03104,SANTYL,0.9538333333333333,MANCHESTER VAMC PHARMACY,718 SMYTH RD,MANCHESTER ...,03104,031,SANTYL
3,MedImpact,718SMYTHRD03104,TORSEMIDE,0.6331666666666667,MANCHESTER VAMC PHARMACY,718 SMYTH RD,MANCHESTER ...,03104,031,TORSEMIDE
4,MedImpact,288SANDOWNRD03826,OMEPRAZOLE,0.3716666666666667,WALGREENS,288 SANDOWN RD,HAMPSTEAD ...,03826,038,OMEPRAZOLE
5,MedImpact,20CRYSTALAVENUE03038,CIPROFLOXACIN,0.5675000000000001,RITE AID PHARMACY 10264,20 CRYSTAL AVENUE,DERRY ...,03038,030,CIPROFLOXACIN HCL
6,MedImpact,17CRYSTALAVE03038,ESTRACE,6.385000000000001,WALGREENS,17 CRYSTAL AVE,DERRY ...,03038,030,ESTRACE
7,MedImpact,TUPPERROAD65RTE6APURITYPLZ02563,BUPROPION,0.6266666666666667,CVS PHARMACY,TUPPER ROAD 65 RTE 6A PURITY PLZ,SANDWICH ...,02563,025,BUPROPION XL
8,MedImpact,861CENTRALST03235,NEOMYCIN-POLYMYXIN-HC,2.6,CVS PHARMACY,861 CENTRAL ST,FRANKLIN ...,03235,032,NEOMYCIN-POLYMYXIN-HC
9,MedImpact,861CENTRALST03235,AMOXICILLIN,0.5428571428571429,CVS PHARMACY,861 CENTRAL ST,FRANKLIN ...,03235,032,AMOXICILLIN


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3283162 entries, 0 to 3283161
Data columns (total 10 columns):
PBMVendor                 object
PharmacyID                object
DrugShortName             object
UnitCost                  object
PharmacyName              object
PharmacyStreetAddress1    object
PharmacyCity              object
PharmacyZip               object
PharmZip                  object
DrugLabelName             object
dtypes: object(10)
memory usage: 250.5+ MB


In [6]:
# convert 'UnitCost' to float
data['UnitCost'] = data['UnitCost'].astype(float)

In [7]:
# clean the white space of Pharmacy City
data['PharmacyCity'] = data['PharmacyCity'].str.strip()

In [8]:
data['PharmacyCity'].head()

0         DERRY
1    MANCHESTER
2    MANCHESTER
3    MANCHESTER
4     HAMPSTEAD
Name: PharmacyCity, dtype: object

In [9]:
# check if we still have any missing values in the dataset
data.isnull().any()

PBMVendor                 False
PharmacyID                False
DrugShortName             False
UnitCost                  False
PharmacyName              False
PharmacyStreetAddress1    False
PharmacyCity              False
PharmacyZip               False
PharmZip                  False
DrugLabelName             False
dtype: bool

In [10]:
# Helper function to return the desirable dataframe given the medication and parent df
def _get_df(df, drug, drug_basic):

    # Check if drug name exists in the pharmacy data, if yes
    # filter the dataset based on that drug name
    if drug in set(df['DrugLabelName'].values):
        df = df[df['DrugLabelName'] == drug]
        return df
    
    # Else, check if broader drug name exists in dataset, if yes
    # filter based on the shorter name
    elif drug_basic in set(df.DrugShortName.values):
        df = df[df['DrugShortName'] == drug_basic]
        return df
    
    else:
        return pd.DataFrame()

In [106]:
# Helper function for get_best_options/get_df to return a sorted dataframe of
# a specified number of entries based on price
# non-mail order

def _cheapest(df, n):
    # Group the input dataframe by pharmacy
    pharmacies = df.groupby(['PharmacyID'], as_index=False).mean()  
    sorted_cost = pharmacies.sort_values(by='UnitCost', ascending=True)
                                         
    sorted_df = pd.DataFrame(sorted_cost, columns=['PharmacyID', 'UnitCost'])
    sorted_df = sorted_df.reset_index(drop=True)
    
    if sorted_df.shape[0] > n:
        sorted_df = sorted_df[:n]
        
    return sorted_df

In [129]:
# Function to retrieve information in the form of data tables with the cheapest
# pharmacies locally and by mail order

def get_best_options(zipcode, drug, number, df):
    
    # Define variables from the input zip code and drug name to match the
    # two engineered columns
    zipcode_short = str(zipcode)[:3]
    drug_basic = drug.split()[0]
    
    # LOCAL PHARMACIES
    # Check if zip code is in the data, then filter by zip code
    if str(zipcode) in set(df.PharmacyZip.values):
        zip_codes = df[df.PharmacyZip==zipcode]
    # Check if broader zip code is in the data, then filter
    elif zipcode_short in set(df.PharmZip.values):
        zip_codes = df[df.PharmZip == zipcode_short]
    else:
        zip_codes = pd.DataFrame()
    
    # Attempt to get cheapest pharmacies for drug
    local_options = _get_df(zip_codes, drug, drug_basic)
    if not local_options.empty:
        local_top = _cheapest(local_options, number)
        local_top['EstimatedPrice'] = local_top.apply(lambda row: '${0:.2f}'.format(number*row.UnitCost), axis=1)
        local_top.drop(columns = ['UnitCost'], inplace=True)
        
        # Get Additional Information For the cheapest N Pharmacies
        the_thing = local_options.PharmacyName[local_options.PharmacyID == local_top.loc[0].iloc[0]]
        local_top['PharmacyName'] = local_top.apply(lambda row: local_options.PharmacyName[local_options.PharmacyID == row.PharmacyID].iloc[0], axis=1)
        local_top['PharmacyStreetAddress1'] = local_top.apply(lambda row: local_options.PharmacyStreetAddress1[local_options.PharmacyID == row.PharmacyID].iloc[0], axis=1)
        local_top['PharmacyZip'] = local_top.apply(lambda row: local_options.PharmacyZip[local_options.PharmacyID == row.PharmacyID].iloc[0], axis=1)
        local_top['PharmacyCity'] = local_top.apply(lambda row: local_options.PharmacyCity[local_options.PharmacyID == row.PharmacyID].iloc[0], axis=1)
    
    # Make series of string to display
    else:
        local_top = pd.Series(data='Unfortunately, we can\'t find any pharmacies in your area, you can always try one of the mail order pharmacies.')
    
    return local_top
    
get_best_options('01908', 'PROAIR', 10, data)

,PharmacyID,EstimatedPrice,PharmacyName,PharmacyStreetAddress1,PharmacyZip,PharmacyCity
0,52BOSTONSTREET01904,$51.71,RITE AID PHARMACY 10129,52 BOSTON STREET,01904,LYNN
1,35EMAINSTREET01960,$53.25,WALGREENS #2208,35 E MAIN STREET,01960,PEABODY
2,85LYNNFIELDSTREET01960,$55.82,CVS PHARMACY,85 LYNNFIELD STREET,01960,PEABODY
3,450HIGHLANDAVENUE01970,$55.82,WAL-MART PHARMACY,450 HIGHLAND AVENUE,01970,SALEM
4,200SOUTHCOMMONST01905,$55.82,CVS PHARMACY,200 SOUTH COMMON ST,01905,LYNN
5,54ELLIOTST01915,$56.13,WALGREENS #4393,54 ELLIOT ST,01915,BEVERLY
6,7WILLOWST01901,$56.45,FLAG PHARMACY,7 WILLOW ST,01901,LYNN
7,166WALNUTST01906,$57.67,WALGREENS #2471,166 WALNUT ST,01906,SAUGUS
8,127135EASTERNAVENUE01930,$58.34,RITE AID PHARMACY 10133,127 135 EASTERN AVENUE,01930,GLOUCESTER
9,272HIGHLANDAVENUE01970,$58.34,CVS PHARMACY,272 HIGHLAND AVENUE,01970,SALEM


In [14]:
# View the most common zip codes for testing ideas
data.PharmacyZip.value_counts()

03756    317047
02128     56820
03246     53344
03301     52579
03431     49844
03820     49445
03060     49312
03038     44227
03784     36671
04401     31746
02150     31185
03743     28579
03264     28475
03063     28311
03110     27672
03103     25705
04011     23907
03109     23813
03257     23555
05819     23075
03867     22827
01702     22731
03051     22439
03053     21717
02740     20992
03458     20949
01453     19890
02151     18635
02130     18452
03755     18366
02149     18166
05301     17561
03054     17357
01757     16987
01844     16980
04901     16727
02155     16331
04412     15850
03102     15553
02148     15162
01608     15092
02301     14031
03801     13701
04330     13617
03055     13203
02118     12806
01082     12700
04210     12531
02125     12294
02745     12119
03878     11868
02143     11861
02145     11776
02119     11710
04240     11664
03773     11600
03860     11432
03104     11288
04530     11216
03106     11173
03079     11044
01851     10615
01604   

In [124]:
# View the most common drug label names for testing ideas
data.DrugLabelName.value_counts()

LISINOPRIL                        41881
PROAIR HFA AER                    29953
FLUTICASONE SPR 50MCG             26595
OMEPRAZOLE                        25971
SIMVASTATIN                       25256
OMEPRAZOLE CAP 20MG               24735
LEVOTHYROXINE SODIUM              23765
ATORVASTATIN CALCIUM              23592
HYDROCHLOROTHIAZIDE               23526
HYDROCHLOROT TAB 25MG             23274
AZITHROMYCIN TAB 250MG            21930
LISINOPRIL TAB 10MG               19918
METFORMIN HCL                     18727
IBUPROFEN                         17985
AMLODIPINE BESYLATE               17785
HYDROCO/APAP TAB 5-325MG          17286
SERTRALINE TAB 100MG              16402
LORAZEPAM TAB 0.5MG               15963
CITALOPRAM TAB 20MG               15738
SERTRALINE TAB 50MG               15582
TRAMADOL HCL TAB 50MG             15439
AMOXICILLIN CAP 500MG             15431
MONTELUKAST TAB 10MG              14886
LISINOPRIL TAB 20MG               14876
SIMVASTATIN TAB 20MG              14512
